# Importante




"""                                                     P/E ou P/L EMPOLGACAO
Conhecido como um múltiplo do preço, pois mostra o quanto os investidores estão dispostos a pagar pelo lucro obtido no período anterior.
"""

"""                                                    EV/EBITDA   ANOS GERACAO DE CAIXA
Da mesma forma que o indicador Preço/Lucro (P/L), quando maior for o EV/EBITDA, mais cara a empresa está no mercado.
Por exemplo, se o EV/EBITDA é igual a 5, isso significa que o valor da empresa (EV) corresponde a cerca de 5 anos de sua geração de caixa operacional (EBITDA).
Para o investidor, quanto menor for esse tempo, mais rapidamente ele consegue colher os frutos dos recursos que aplicou.
"""

"""                                                    EV/REVENUE subvalorizada
O múltiplo valor/receita (EV/R) da empresa ajuda a comparar as receitas de uma empresa com o valor da empresa.
Quanto menor, melhor, pois um múltiplo EV/R menor sinaliza que uma empresa está subvalorizada.
"""

"""                                                     P/VP
A razão de preço para livro (relação P/B) é um índice financeiro usado para comparar o valor contábil de uma empresa com o preço atual de mercado.
É calculado dividindo o preço de fechamento atual das ações pelo valor contábil do último trimestre por ação. Quanto menor a razão de preço para livro, melhor o valor
"""


"""                                 P/ToSales12   CAPACIDADE GERAR RECEITA
ao analisar o PSR é possível concluir que empresas com índice menor que 1,0 conseguem gerar mais receita em comparação com seu valor de mercado.

"""



"""                                                     P/E ou P/L EMPOLGACAO
Conhecido como um múltiplo do preço, pois mostra o quanto os investidores estão dispostos a pagar pelo lucro obtido no período anterior.
"""

"""                                                    EV/EBITDA   ANOS GERACAO DE CAIXA
Da mesma forma que o indicador Preço/Lucro (P/L), quando maior for o EV/EBITDA, mais cara a empresa está no mercado.
Por exemplo, se o EV/EBITDA é igual a 5, isso significa que o valor da empresa (EV) corresponde a cerca de 5 anos de sua geração de caixa operacional (EBITDA).
Para o investidor, quanto menor for esse tempo, mais rapidamente ele consegue colher os frutos dos recursos que aplicou.
"""

"""                                                    EV/REVENUE subvalorizada
O múltiplo valor/receita (EV/R) da empresa ajuda a comparar as receitas de uma empresa com o valor da empresa.
Quanto menor, melhor, pois um múltiplo EV/R menor sinaliza que uma empresa está subvalorizada.
"""

"""                                                     P/VP
A razão de preço para livro (relação P/B) é um índice financeiro usado para comparar o valor contábil de uma empresa com o preço atual de mercado.
É calculado dividindo o preço de fechamento atual das ações pelo valor contábil do último trimestre por ação. Quanto menor a razão de preço para livro, melhor o valor
"""


"""                                 P/ToSales12   CAPACIDADE GERAR RECEITA
ao analisar o PSR é possível concluir que empresas com índice menor que 1,0 conseguem gerar mais receita em comparação com seu valor de mercado.

"""




# INSTALL

In [142]:
!pip install pandas yfinance numpy matplotlib IPython

# Import

In [143]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import math
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display



# Dados

In [144]:


# https://github.com/ranaroussi/yfinance
# https://pypi.org/project/yfinance/

class ACAO():

    def __init__(self,quantidade_de_acao_prospectada,ativos,CARTEIRA,valor_disponivel) -> None:
        self.ativos=ativos
        self.quantidade_de_acao_prospectada = quantidade_de_acao_prospectada
        self.carteira=CARTEIRA
        self.valor_disponivel = valor_disponivel

    def start(self):
        pd.reset_option('max_columns')
        df =pd.DataFrame([],columns=['VALOR_PROSPECTADO_DIV_MEDIO','VALO_ACAO/DIVIDENDO','VALOR_REAL_DIV_MEDIO','ACAO','PRECO','PRECOJUSTO','PRECOTETO_10','DIV.MEDIO','P/L','LPA','P/VP','P/ToSales12','EV/EBITDA','EV/REVENUE','VARIACAO'])
        for x,y in enumerate(self.ativos):
            self.get_dados(x,y,df)
        df = df.sort_values(['EV/EBITDA','EV/REVENUE','P/ToSales12','P/VP'], ascending=[True,True,True,True])
        df['PRECO/PRECOTETO_10'] = df['PRECO'] - df['PRECOTETO_10']
        df = df.sort_values(['PRECO/PRECOTETO_10','EV/EBITDA','EV/REVENUE','P/ToSales12','P/VP'], ascending=[True,True,True,True,True])
        return df

    def get_dados(self,index,acao,df):

        aapl = yf.Ticker(f"{acao}.SA")
        pd.set_option('display.max_rows', None)
        VPA=aapl.info['bookValue']
        PL = aapl.info['trailingPE']
        LPA =aapl.info['trailingEps']
        num= (2.25 * PL* VPA*LPA)
        preco_justo = math.pow(num, 1/2)
        dividendo_medio_5= pd.Series(aapl.dividends.resample('Y').sum()).tail(3).mean()         #3 anos
        preco_teto = round(dividendo_medio_5/0.06,2)
        df.loc[index,'ACAO']=acao
        df.loc[index,'PRECO']=aapl.info['currentPrice']
        df.loc[index,'VARIACAO']= round(aapl.info['52WeekChange']*100,2) if "52WeekChange" in aapl.info else 0.0
        df.loc[index,'PRECOJUSTO']= round(preco_justo,2)
        df.loc[index,'DIV.MEDIO']=round(float(dividendo_medio_5),2)
        df.loc[index,'VALOR_PROSPECTADO_DIV_MEDIO'] =   round((self.quantidade_de_acao_prospectada/aapl.info['currentPrice']) * dividendo_medio_5,2)
        df.loc[index,'VALOR_REAL_DIV_MEDIO']=round(self.carteira[index]*dividendo_medio_5,2)
        df.loc[index,'PRECOTETO_10']=round(float(dividendo_medio_5)/0.1,2)
        df.loc[index,'VALO_ACAO/DIVIDENDO']= round(round(self.carteira[index]*dividendo_medio_5,2)/round(aapl.info['currentPrice']*self.quantidade_de_acao_prospectada,2)*100,2)
        df.loc[index,'P/VP']= round(aapl.info['priceToBook'],2)
        df.loc[index,'P/ToSales12']= round(aapl.info['priceToSalesTrailing12Months'],2)
        df.loc[index,'P/L']= PL
        df.loc[index,'LPA']= LPA
        df.loc[index,'EV/EBITDA']= round(aapl.info['enterpriseToEbitda'],2) if "enterpriseToEbitda" in aapl.info else 0.0
        df.loc[index,'EV/REVENUE']= round(aapl.info['enterpriseToRevenue'],2) if "enterpriseToRevenue" in aapl.info else 0.0

    def lucro_por_preco(self,acao):
        aapl = yf.Ticker(f"{acao}.SA")
        dx= aapl.financials
        ########################################################################################
        lucro_liquido= dx[dx.index.isin(['Net Income'])]
        lucro_liquido =lucro_liquido.transpose()
        lucro_liquido=lucro_liquido.infer_objects(copy=False).fillna(0)# type: ignore
        lucro_liquido.index.name = 'Date'
        lucro_liquido.index= pd.to_datetime(lucro_liquido.index).date
        ########################################################################################
        dp=aapl.history(period="5y").reset_index()
        dp['Date'] = pd.to_datetime(dp['Date'])
        dp=dp.resample(on="Date", rule="ME").max().reset_index(drop=False)
        dp['year'] = dp['Date'].dt.year
        dp['month'] = dp['Date'].dt.month
        dp['Date'] =  dp['Date'].dt.date
        dp1 = dp[dp['month']==12]
        # dp1.loc[:, ['Close']] =dp1['Close'] * 10000000
        dp2 = dp1[['Date','Close']].set_index(['Date'])
        ########################################################################################
        dx=pd.concat([dp2,lucro_liquido],axis=1 ).reset_index()
        dx=dx.infer_objects(copy=False).fillna(0) # type: ignore
        dx.rename(columns={'index': 'Data', 'Close': 'Preco','Net Income':'Lucro'}, inplace=True)
        dx.sort_values(['Data'], ascending=[False])

        fig, ax1 = plt.subplots(1,1,num=acao)
        dx.Preco.plot(ax=ax1, color='blue', label='Preco')
        ax2 = ax1.twinx()
        dx.Lucro.plot(ax=ax2, color='green', label='Lucro')
        ax1.set_ylabel('Preco')
        ax2.set_ylabel('Lucro')
        ax1.legend(loc=3)
        ax2.legend(loc=0) # type: ignore
        ax1.set_xticklabels(dx.Data)
        ax1.set_xticks(np.arange(len(dx.Data)))
        # plt.savefig(f'{acao}.png')

    def GRAFICO(self,dx,tag):
        dx.plot(kind="bar",x="ACAO",y=['PRECO','VALOR_COMPRA'])
        # plt.show()
        plt.savefig(f'{tag}.png')

CARTEIRA = {
    "CXSE3":0,
    "BBSE3":0,
    "BBAS3":1500,
    "TAEE11":600,
    "UNIP6":0,
    "TRPL4":0,
    "VALE3":700,
    "BBDC4":0,
    "SAPR11":0,
    "CSMG3":0,
    "SANB11":0,
    "ITSA4":0,
    "CMIG4":0,
    "RANI3":0,
    "KLBN11":0,
    "PETR4":0,
    "BBDC4":0,
    "BRSR6":0,
    "GGBR4":0,
    "CMIN3":0,
    "BRAP4":0,
    "CPFE3":0,
    "LEVE3":0
}


# CARTEIRA = {
#     "VALE3":0
# }
ACOES = list(CARTEIRA.keys())
VALORES = list(CARTEIRA.values())
# VALORES = [1000 for x in range(0,9)]
valor = 1000
valor_prospectado = valor
valor_disponivel = valor #float(input("Digite Valor Total Disponivel: "))
df = ACAO(valor_prospectado,ACOES,VALORES,valor_disponivel).start()


# SHOW

In [149]:
def valida_acao(x):
              compra = 'background-color:green ;color:white'
              atencao = 'background-color:yellow ;color:back'
              venda =  'background-color:blue ;color:white'

              default=''
              if  x['EV/EBITDA']<=10.0 and x['P/ToSales12']<=10.0 and x['PRECO'] - x['PRECOTETO_10'] >= -10.00 and x['PRECO'] - x['PRECOTETO_10'] < 0.00 and x['VARIACAO']<=0 :
                  return [compra,compra,compra,compra,compra,compra]
              if  x['EV/EBITDA']<=10.0 and x['P/ToSales12']<=10.0 and x['PRECO'] - x['PRECOTETO_10'] >= 0.00 and x['PRECO'] - x['PRECOTETO_10'] < 2.00 :
                  return [atencao,atencao,atencao,atencao,atencao,atencao]
              if  x['EV/EBITDA']<=10.0 and x['P/ToSales12']<=10.0 and x['PRECO'] - x['PRECOTETO_10'] >= 2.00 and x['VARIACAO']>=0 :
                  return [venda,venda,venda,venda,venda,venda]
              return [default,default,default,default,default,default]
df.style.apply(valida_acao,subset=['ACAO','PRECO','EV/EBITDA','P/ToSales12','PRECOTETO_10','VARIACAO'],axis=1)


,VALOR_PROSPECTADO_DIV_MEDIO,VALO_ACAO/DIVIDENDO,VALOR_REAL_DIV_MEDIO,ACAO,PRECO,PRECOJUSTO,PRECOTETO_10,DIV.MEDIO,P/L,LPA,P/VP,P/ToSales12,EV/EBITDA,EV/REVENUE,VARIACAO,PRECO/PRECOTETO_10
15,238.760000,0.000000,0.000000,PETR4,38.350000,50.000000,91.560000,9.160000,5.470756,7.010000,1.320000,1.040000,3.390000,1.530000,14.980000,-53.210000
21,126.920000,0.000000,0.000000,LEVE3,30.790000,22.540000,39.080000,3.910000,6.220202,4.950000,4.200000,0.970000,6.020000,1.160000,-34.670000,-8.290000
4,110.630000,0.000000,0.000000,UNIP6,48.460000,50.690000,53.610000,5.360000,12.786280,3.790000,2.060000,1.230000,8.830000,1.380000,-32.390000,-5.150000
19,125.560000,0.000000,0.000000,BRAP4,18.870000,30.210000,23.690000,2.370000,5.360796,3.520000,0.880000,459.270000,0.000000,448.160000,-19.500000,-4.820000
18,105.600000,0.000000,0.000000,CMIN3,5.950000,5.350000,6.280000,0.630000,7.083334,0.840000,2.780000,1.810000,3.940000,1.590000,33.800000,-0.330000
12,102.390000,0.000000,0.000000,CMIG4,11.910000,15.810000,12.190000,1.220000,6.170985,1.930000,1.280000,0.970000,4.920000,1.130000,28.280000,-0.280000
6,98.350000,6.880000,3949.760000,VALE3,57.370000,78.800000,56.430000,5.640000,4.513769,12.710000,1.190000,1.170000,3.710000,1.490000,-16.180000,0.940000
13,84.550000,0.000000,0.000000,RANI3,7.800000,9.700000,6.600000,0.660000,12.187501,0.640000,1.450000,1.190000,7.010000,1.860000,-27.300000,1.200000
5,91.000000,0.000000,0.000000,TRPL4,25.110000,39.940000,22.850000,2.290000,6.200000,4.050000,0.890000,2.690000,7.800000,3.810000,4.430000,2.260000
17,85.340000,0.000000,0.000000,GGBR4,18.350000,32.890000,15.660000,1.570000,7.677824,2.390000,0.700000,0.580000,5.370000,0.710000,-13.790000,2.690000
